In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Hull_df = pd.read_csv('Hull15-22-final.csv')

In [3]:
def shift_columns(df):
    # Shift the PM2.5 column values by 1, 2, and 3 positions
    df['1 hour'] = df['PM2.5'].shift(-1)
    df['2 hour'] = df['PM2.5'].shift(-2)
    df['3 hour'] = df['PM2.5'].shift(-3)
    df = df.iloc[:-3]
    return df

In [5]:
hull_df = shift_columns(Hull_df)

In [6]:
scale_factors = {
   'Ozone':10, 'Hourly Total Radiation': 100,'Cloud Base Height':100, 'Humidity':10
}

# Apply scaling
for column, factor in scale_factors.items():
    if column in hull_df.columns:
        hull_df[column] = hull_df[column] / factor

C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\2520987080.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hull_df[column] = hull_df[column] / factor


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [8]:
 features = ['Nitric oxide', 'Nitrogen dioxide', 'Nitrogen oxides', 'PM10','PM2.5','Temperature', 'Wind Speed',
              'Humidity', 'Wind Cardinal Direction', 'Maximum Gust Speed', 'Total Cloud Amount',
           'Ozone', 'Hourly Total Radiation','Cloud Base Height','hour', 'season']

In [9]:
def clear_model_memory():
    #Clear the memory of the model
  
    # Clear memory for scikit-learn
    gc.collect()
    
    # Clear memory for Keras/TensorFlow
    tf.keras.backend.clear_session()
    gc.collect()

In [10]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed
from tensorflow.keras.optimizers import Adam,  RMSprop
from keras.callbacks import EarlyStopping


In [11]:
def create_sequences(data, target, time_steps=3):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(target[i + time_steps])
    return np.array(X), np.array(y)

In [12]:
winter_df = pd.read_csv('winter_data.csv')
spring_df = pd.read_csv('spring_data.csv')
summer_df = pd.read_csv('summer_data.csv')
fall_df = pd.read_csv('autumn_data.csv')

In [14]:
defs = [winter_df, spring_df, summer_df, fall_df]
# Apply scaling
for df in defs:
    for column, factor in scale_factors.items():
        if column in df.columns:
            df[column] = df[column] / factor

def shift_column(df):
    # Shift the PM2.5 column values by 1, 2, and 3 positions
    df['1 hour'] = df['PM2.5'].shift(-1)
    df['2 hour'] = df['PM2.5'].shift(-2)
    df['3 hour'] = df['PM2.5'].shift(-3)
    return df

for i in range(len(defs)):
    defs[i] = shift_column(defs[i])
 # Drop the last 3 rows
for i in range(len(defs)):
    defs[i] =defs[i].iloc[:-3]    

In [17]:
scale_factors = {
   'Ozone':10, 'Hourly Total Radiation': 100,'Cloud Base Height':100, 'Humidity':10
}

# Apply scaling
for column, factor in scale_factors.items():
    if column in hull_df.columns:
        hull_df[column] = hull_df[column] / factor

C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\2520987080.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hull_df[column] = hull_df[column] / factor


In [18]:
def create_sequences(data, target, time_steps=3):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(target[i + time_steps])
    return np.array(X), np.array(y)

In [19]:
def predict_pm25_cnn_lstm(df, features, epochs=50, batch_size=64, time_steps=3, output_folder="results"):
    # Ensure 'Date and time' is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(df['Date and time']):
        df['Date and time'] = pd.to_datetime(df['Date and time'])
    
    # Sorting by datetime to maintain order
    df = df.sort_values(by='Date and time')
    
    metrics_table = []
    
    # List of target columns
    targets = ['1 hour']
    
    for target in targets:
        # Preparing the dataset
        X = df[features].values
        y = df[target].values
        
        # Standardize the features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Create sequences
        X_seq, y_seq = create_sequences(X_scaled, y, time_steps)
        
        # Reshape for Conv1D
        X_seq = X_seq.reshape((X_seq.shape[0], time_steps, X_seq.shape[2], 1))
        
        # Splitting the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
        
        # Creating and training the CNN-LSTM model
        model = Sequential()
        model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2, activation='relu'), input_shape=(time_steps, X_train.shape[2], 1)))
        model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
        model.add(TimeDistributed(Flatten()))
        model.add(LSTM(50, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(LSTM(50))
        model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
        
        # Early stopping to avoid overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1, shuffle=False, callbacks=[early_stopping])
        
        # Predicting the PM2.5 values for the test set
        y_pred = model.predict(X_test).flatten()  # Flatten to ensure y_pred is 1D
        
        # Calculating performance metrics
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        
        # Append metrics to the table
        metrics_table.append({
            'Target': target,
            'MSE': mse,
            'MAE': mae,
            'R-squared': r2,
            'MAPE': mape
        })
        
        # Predicting PM2.5 for each row in the DataFrame
        X_full_seq, _ = create_sequences(X_scaled, y, time_steps)
        X_full_seq = X_full_seq.reshape((X_full_seq.shape[0], time_steps, X_full_seq.shape[2], 1))
        predictions = model.predict(X_full_seq).flatten()
        
        # Add NaN for the first time_steps values to match the original DataFrame length
        predictions = np.concatenate([np.full(time_steps, np.nan), predictions])
        df[f'PM2.5_pre_{target}_cnn_lstm'] = predictions
    
    # Convert metrics table to DataFrame for better visualization
    metrics_df = pd.DataFrame(metrics_table)
    
    # Save the metrics DataFrame and the updated dataset
    metrics_df.to_csv(f'{output_folder}/metrics_table_cnn_lstm.csv', index=False)
    df.to_csv(f'{output_folder}/predicted_pm25_cnn_lstm.csv', index=False)
    
    print("Metrics Table:")
    print(metrics_df)
    return df, metrics_df


In [16]:
from sklearn.model_selection import ParameterGrid
import itertools

def fine_tune_cnn_lstm(df, features, param_grid, time_steps=3, output_folder="results"):
    best_score = -np.inf
    best_params = None
    results = []
    
    # Generate all combinations of hyperparameters
    param_combinations = list(itertools.product(
        param_grid['epochs'],
        param_grid['batch_size'],
        param_grid['learning_rate'],
        param_grid['lstm_units'],
        param_grid['dropout_rate']
    ))
    
    for combination in param_combinations:
        epochs, batch_size, learning_rate, lstm_units, dropout_rate = combination
        print(f"Testing combination: Epochs={epochs}, Batch={batch_size}, Learning Rate={learning_rate}, LSTM Units={lstm_units}, Dropout={dropout_rate}")
        
        try:
            df_result, metrics_df = predict_pm25_cnn_lstm(
                df=df,
                features=features,
                epochs=epochs,
                batch_size=batch_size,
                time_steps=time_steps,
                output_folder=output_folder
            )
        except Exception as e:
            print(f"Error occurred during combination {combination}: {e}")
            continue
        
        # Extract R-squared value for '1 hour' target
        r2_value = metrics_df.loc[metrics_df['Target'] == '1 hour', 'R-squared'].values[0]
        
        # Save results and update best score if applicable
        results.append((combination, r2_value))
        if r2_value > best_score:
            best_score = r2_value
            best_params = combination

    print(f"Best parameters: {best_params} with score: {best_score}")
    return best_params, best_score, results




In [21]:
# Example hyperparameter grid
features = ['PM10','PM2.5', 'Visibility', 'Ozone', 'Nitrogen dioxide', 'Nitrogen oxides', 'Nitric oxide']
param_grid = {
    'epochs': [10],
    'batch_size': [32, 64],
    'learning_rate': [0.001, 0.0001],
    'lstm_units': [50, 100],
    'dropout_rate': [0.2, 0.3]
}

# Fine-tune the CNN-LSTM model
best_params, best_score, results = fine_tune_cnn_lstm(df=hull_df, features=features, param_grid=param_grid)

Testing combination: Epochs=10, Batch=32, Learning Rate=0.001, LSTM Units=50, Dropout=0.2


C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\1548625060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date and time'] = pd.to_datetime(df['Date and time'])




Epoch 1/10

1754/1754 [==============================] - 24s 9ms/step - loss: 31.0342 - val_loss: 21.1333
Epoch 2/10
1754/1754 [==============================] - 15s 9ms/step - loss: 21.2304 - val_loss: 19.7892
Epoch 3/10
1754/1754 [==============================] - 15s 9ms/step - loss: 20.5477 - val_loss: 19.3992
Epoch 4/10
1754/1754 [==============================] - 15s 9ms/step - loss: 20.1883 - val_loss: 19.2635
Epoch 5/10
1754/1754 [==============================] - 15s 9ms/step - loss: 19.9651 - val_loss: 19.2330
Epoch 6/10
1754/1754 [==============================] - 15s 9ms/step - loss: 19.8231 - val_loss: 19.1205
Epoch 7/10
1754/1754 [==============================] - 15s 9ms/step - loss: 19.7053 - val_loss: 19.0244
Epoch 8/10
1754/1754 [==============================] - 15s 9ms/step - loss: 19.4723 - val_loss: 18.8466
Epoch 9/10
1754/1754 [==============================] - 15s 9ms/step - loss: 19.3764 - val_loss: 18.8557
Epoch 10/10
2192/2192 [=============================

In [22]:
param_grid = {
    'epochs': [50],
    'batch_size': [64],
    'learning_rate': [0.0001],
    'lstm_units': [100],
    'dropout_rate': [0.2]
}
best_params, best_score, results = fine_tune_cnn_lstm(df=hull_df, features=features, param_grid=param_grid)

Testing combination: Epochs=50, Batch=64, Learning Rate=0.0001, LSTM Units=100, Dropout=0.2
Epoch 1/50
877/877 [==============================] - 16s 10ms/step - loss: 38.4169 - val_loss: 22.9786
Epoch 2/50
877/877 [==============================] - 9s 10ms/step - loss: 22.4135 - val_loss: 20.3301
Epoch 3/50
877/877 [==============================] - 9s 10ms/step - loss: 20.9188 - val_loss: 19.7878
Epoch 4/50
877/877 [==============================] - 9s 10ms/step - loss: 20.2204 - val_loss: 19.3245
Epoch 5/50
877/877 [==============================] - 9s 10ms/step - loss: 19.9489 - val_loss: 19.2424
Epoch 6/50
877/877 [==============================] - 9s 10ms/step - loss: 19.8714 - val_loss: 19.2294
Epoch 7/50
877/877 [==============================] - 9s 10ms/step - loss: 19.6679 - val_loss: 19.0207
Epoch 8/50
877/877 [==============================] - 9s 10ms/step - loss: 19.4849 - val_loss: 19.1970
Epoch 9/50
877/877 [==============================] - 9s 10ms/step - loss: 19.4086 

In [23]:
 features = ['Nitric oxide', 'Nitrogen dioxide', 'Nitrogen oxides', 'PM10','PM2.5','Temperature', 'Wind Speed',
              'Humidity', 'Wind Cardinal Direction', 'Maximum Gust Speed', 'Total Cloud Amount',
           'Ozone', 'Hourly Total Radiation','Cloud Base Height','hour', 'season']
best_params, best_score, results = fine_tune_cnn_lstm(df=hull_df, features=features, param_grid=param_grid)

Testing combination: Epochs=50, Batch=64, Learning Rate=0.0001, LSTM Units=100, Dropout=0.2
Epoch 1/50
877/877 [==============================] - 16s 10ms/step - loss: 37.9627 - val_loss: 22.7678
Epoch 2/50
877/877 [==============================] - 9s 11ms/step - loss: 22.2132 - val_loss: 20.0637
Epoch 3/50
877/877 [==============================] - 10s 11ms/step - loss: 20.3750 - val_loss: 19.2723
Epoch 4/50
877/877 [==============================] - 11s 13ms/step - loss: 19.8219 - val_loss: 19.0250
Epoch 5/50
877/877 [==============================] - 11s 13ms/step - loss: 19.4392 - val_loss: 18.7065
Epoch 6/50
877/877 [==============================] - 9s 11ms/step - loss: 19.1819 - val_loss: 18.7290
Epoch 7/50
877/877 [==============================] - 10s 11ms/step - loss: 18.9177 - val_loss: 18.4564
Epoch 8/50
877/877 [==============================] - 10s 11ms/step - loss: 18.6109 - val_loss: 18.5178
Epoch 9/50
877/877 [==============================] - 10s 11ms/step - loss: 18

In [24]:
best_params, best_score, results = fine_tune_cnn_lstm(df=defs[0], features=features, param_grid=param_grid)

Testing combination: Epochs=50, Batch=64, Learning Rate=0.0001, LSTM Units=100, Dropout=0.2


C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\1548625060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date and time'] = pd.to_datetime(df['Date and time'])


Epoch 1/50
217/217 [==============================] - 11s 16ms/step - loss: 82.1556 - val_loss: 55.2341
Epoch 2/50
217/217 [==============================] - 3s 12ms/step - loss: 41.1650 - val_loss: 35.2504
Epoch 3/50
217/217 [==============================] - 2s 11ms/step - loss: 31.0564 - val_loss: 28.3592
Epoch 4/50
217/217 [==============================] - 2s 11ms/step - loss: 26.7758 - val_loss: 25.2611
Epoch 5/50
217/217 [==============================] - 3s 12ms/step - loss: 24.4577 - val_loss: 23.5000
Epoch 6/50
217/217 [==============================] - 3s 12ms/step - loss: 22.9506 - val_loss: 22.2442
Epoch 7/50
217/217 [==============================] - 3s 12ms/step - loss: 21.9818 - val_loss: 21.6447
Epoch 8/50
217/217 [==============================] - 3s 14ms/step - loss: 21.4047 - val_loss: 20.8131
Epoch 9/50
217/217 [==============================] - 3s 12ms/step - loss: 21.0214 - val_loss: 21.1668
Epoch 10/50
217/217 [==============================] - 3s 14ms/step - lo

In [25]:
best_params, best_score, results = fine_tune_cnn_lstm(df=defs[1], features=features, param_grid=param_grid)

Testing combination: Epochs=50, Batch=64, Learning Rate=0.0001, LSTM Units=100, Dropout=0.2


C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\1548625060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date and time'] = pd.to_datetime(df['Date and time'])


Epoch 1/50
221/221 [==============================] - 11s 20ms/step - loss: 87.7533 - val_loss: 56.5111
Epoch 2/50
221/221 [==============================] - 3s 15ms/step - loss: 47.3591 - val_loss: 37.1419
Epoch 3/50
221/221 [==============================] - 4s 16ms/step - loss: 35.8884 - val_loss: 30.1434
Epoch 4/50
221/221 [==============================] - 4s 17ms/step - loss: 31.1644 - val_loss: 26.9216
Epoch 5/50
221/221 [==============================] - 4s 16ms/step - loss: 28.3546 - val_loss: 25.1344
Epoch 6/50
221/221 [==============================] - 4s 16ms/step - loss: 26.8566 - val_loss: 24.4641
Epoch 7/50
221/221 [==============================] - 3s 16ms/step - loss: 26.0904 - val_loss: 24.4340
Epoch 8/50
221/221 [==============================] - 4s 17ms/step - loss: 24.8704 - val_loss: 23.6101
Epoch 9/50
221/221 [==============================] - 4s 16ms/step - loss: 24.4338 - val_loss: 23.2037
Epoch 10/50
221/221 [==============================] - 4s 16ms/step - lo

In [26]:
best_params, best_score, results = fine_tune_cnn_lstm(df=defs[2], features=features, param_grid=param_grid)

Testing combination: Epochs=50, Batch=64, Learning Rate=0.0001, LSTM Units=100, Dropout=0.2


C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\1548625060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date and time'] = pd.to_datetime(df['Date and time'])


Epoch 1/50
221/221 [==============================] - 10s 22ms/step - loss: 52.4601 - val_loss: 33.1962
Epoch 2/50
221/221 [==============================] - 4s 18ms/step - loss: 28.8349 - val_loss: 23.7570
Epoch 3/50
221/221 [==============================] - 4s 18ms/step - loss: 23.7113 - val_loss: 21.0393
Epoch 4/50
221/221 [==============================] - 4s 18ms/step - loss: 21.1810 - val_loss: 19.5981
Epoch 5/50
221/221 [==============================] - 4s 18ms/step - loss: 20.3339 - val_loss: 18.0350
Epoch 6/50
221/221 [==============================] - 4s 18ms/step - loss: 19.5467 - val_loss: 18.0347
Epoch 7/50
221/221 [==============================] - 4s 18ms/step - loss: 19.0639 - val_loss: 17.4411
Epoch 8/50
221/221 [==============================] - 4s 18ms/step - loss: 18.4892 - val_loss: 16.7868
Epoch 9/50
221/221 [==============================] - 4s 18ms/step - loss: 18.1258 - val_loss: 16.8134
Epoch 10/50
221/221 [==============================] - 4s 19ms/step - lo

In [27]:
best_params, best_score, results = fine_tune_cnn_lstm(df=defs[3], features=features, param_grid=param_grid)

Testing combination: Epochs=50, Batch=64, Learning Rate=0.0001, LSTM Units=100, Dropout=0.2


C:\Users\nassi\AppData\Local\Temp\ipykernel_22080\1548625060.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date and time'] = pd.to_datetime(df['Date and time'])


Epoch 1/50
219/219 [==============================] - 9s 12ms/step - loss: 58.1205 - val_loss: 35.3532
Epoch 2/50
219/219 [==============================] - 2s 8ms/step - loss: 29.4109 - val_loss: 23.7194
Epoch 3/50
219/219 [==============================] - 2s 8ms/step - loss: 23.4543 - val_loss: 20.7134
Epoch 4/50
219/219 [==============================] - 2s 8ms/step - loss: 21.3233 - val_loss: 19.3798
Epoch 5/50
219/219 [==============================] - 2s 8ms/step - loss: 19.8850 - val_loss: 18.6263
Epoch 6/50
219/219 [==============================] - 2s 8ms/step - loss: 19.4351 - val_loss: 18.2687
Epoch 7/50
219/219 [==============================] - 2s 8ms/step - loss: 19.1410 - val_loss: 17.7367
Epoch 8/50
219/219 [==============================] - 2s 8ms/step - loss: 18.8651 - val_loss: 17.5349
Epoch 9/50
219/219 [==============================] - 2s 8ms/step - loss: 18.1532 - val_loss: 17.2346
Epoch 10/50
219/219 [==============================] - 2s 8ms/step - loss: 17.933